## Determine pricing and get metadata into recognition task experiment directories

In [ ]:
import os
import csv
import time
import socket
import random
import pandas as pd
from PIL import Image

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('..')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))
gallery_dir = os.path.abspath(os.path.join(proj_dir,'gallery'))
if socket.gethostname() == 'nightingale':
    feature_dir = os.path.abspath('/mnt/pentagon/photodraw/features/')
else:
    feature_dir = os.path.abspath(os.path.join(proj_dir,'features'))
    
meta_path = os.path.abspath(os.path.join(feature_dir, 'metadata_pixels.csv'))
image_path = os.path.abspath(os.path.join(feature_dir, 'flattened_sketches_pixels.npy'))
meta_path_fc6 = os.path.abspath(os.path.join(feature_dir, 'METADATA_sketch.csv'))
image_path_fc6 = os.path.abspath(os.path.join(feature_dir, 'FEATURES_FC6_sketch_no-channel-norm.npy'))

In [ ]:
K = pd.read_csv(os.path.join(csv_dir, 'photodraw2x2_sketch_data.csv'))

### Determine pricing & setup for recognition experiments
#### Category-level recognition task setup

In [ ]:
# experiment parameters
pay_per_hour = 12
seconds_per_rating = 3.5580
catch_trials = 4
ratings_per_session = 32 * 4 + 4
ratings_per_sketch = 3
instruct_surveys_length = 4

In [ ]:
num_sketches_overall = len(K)

total_session_length = ratings_per_session + catch_trials

total_ratings = num_sketches_overall * ratings_per_sketch
total_participants_needed = total_ratings / ratings_per_session
session_time = (total_session_length * seconds_per_rating / 60) + instruct_surveys_length
total_pay_hours = total_participants_needed * session_time / 60
total_pay = total_pay_hours * pay_per_hour
print(f'category level recognition task: \n \n'
      f'total pay: ${total_pay} \n'
      f'total cost: ${total_pay * 1.33} \n'
      f'total participants: {int(total_participants_needed)} \n'
      f'experiment length: {session_time} mins \n'
      f'session length: {total_session_length} trials \n'
      f'assuming {seconds_per_rating} seconds per rating \n'
      f'{ratings_per_sketch} rating(s) per sketch')

#### Instance-level recognition task setup

In [ ]:
# experiment parameters
pay_per_hour = 12
seconds_per_rating = 9.39355
catch_trials = 4
ratings_per_session = 32 * 4 + 4
ratings_per_sketch = 3

In [ ]:
num_sketches_photo = len(K[K.condition == 'photo'])

total_session_length = ratings_per_session + catch_trials

total_ratings = num_sketches_photo * ratings_per_sketch
total_participants_needed = total_ratings / ratings_per_session
session_time = (total_session_length * seconds_per_rating / 60) + instruct_surveys_length
total_pay_hours = total_participants_needed * session_time / 60
total_pay = total_pay_hours * pay_per_hour
print(f'instance level recognition task: \n \n'
      f'total pay: ${total_pay} \n'
      f'total cost: ${total_pay * 1.33} \n'
      f'total participants: {int(total_participants_needed)} \n'
      f'experiment length: {session_time} mins \n'
      f'session length: {total_session_length} trials \n'
      f'assuming {seconds_per_rating} seconds per rating \n'
      f'{ratings_per_sketch} rating(s) per sketch')

### Get metadata files in .js format for each experiment

#### Category-level recognition task

In [ ]:
photodraw32_meta = pd.read_csv('photodraw32_metadata.csv')
photodraw32_sketches_meta = pd.read_csv('photodraw32_s3_sketches_metadata.csv')

In [ ]:
for cat in photodraw32_sketches_meta.category.unique():
    cat_subset = photodraw32_sketches_meta[photodraw32_sketches_meta.category == cat]
    for goal in cat_subset.goal.unique():
        cat_goal_subset = cat_subset[cat_subset.goal == goal]
        for condition in cat_goal_subset.condition.unique():
            cat_goal_condition_subset = cat_goal_subset[cat_goal_subset.condition == condition]
            inds = cat_goal_condition_subset.index.values
            photodraw32_sketches_meta.loc[inds, 'batch_num'] = random.sample(range(len(inds)), len(inds))

photodraw32_sketches_meta['batch_num'] = photodraw32_sketches_meta.batch_num.astype(int)
if False:
    photodraw32_sketches_meta.to_csv('photodraw32_category_recog.csv', index=False)
    with open('photodraw32_category_recog.csv') as f:
        a = [{k: v for k, v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)]
    print(a,  file=open(os.path.join(exp_dir, 'recogdraw_category', 'photodraw32_category_recog.js'), 'w'))

#### Instance-level recognition task

In [ ]:
def check_for_repeats():
    for i in range(int(len(batches) / 3)):
        if any([j[0] == j[1] for j in combinations(batches[i*3:(i+1)*3], 2)]):
            return False
    return True

In [ ]:
instance_stims_meta = pd.read_csv('photodraw32_instance_validation_metadata.csv')

In [ ]:
for cat in instance_stims_meta.category.unique():
    cat_subset = instance_stims_meta[instance_stims_meta.category == cat]
    
    for goal in cat_subset.goal.unique():
        cat_goal_subset = cat_subset[cat_subset.goal == goal]
        inds = cat_goal_subset.index.values
    
        batches = list(range(int(len(inds) / 2))) * 2
        random.shuffle(batches)
        # make sure no three adjacent elements have the same batch 
        while check_for_repeats():
            random.shuffle(batches)

        for part, img in enumerate(cat_goal_subset.image_id.unique()):
            cat_goal_img_subset = cat_goal_subset[cat_goal_subset.image_id == img]
            inds = cat_goal_img_subset.index.values
            three_batch = [batches.pop(0) for i in range(3)]
            instance_stims_meta.loc[inds, 'batch_num'] = three_batch

instance_stims_meta['batch_num'] = instance_stims_meta.batch_num.astype(int)

if False:
    instance_stims_meta.to_csv('photodraw32_instance_recog.csv', index=False)
    with open('photodraw32_instance_recog.csv') as f:
        a = [{k: v for k, v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)]
    print(a,  file=open(os.path.join(exp_dir, 'recogdraw_instance', 'photodraw32_instance_recog.js'), 'w'))